In [43]:
import os
import h5py
import gvar   as gv
import numpy  as np
import pandas as pd
from   matplotlib import pyplot as plt

def RatioFileString(name,ts,h,hss,q,l,p):
    return f'{name}T{ts}{h}_RW_{hss}_rot_rot{q}{l}_p{p}'

def format(v):
    return gv.gvar(
        v.mean(axis=0),
        v.std(axis=0) * np.sqrt(v.shape[0]-1)
    )

def plot(v):
    plt.errorbar(
        np.arange(len(v)),
        gv.mean(v),
        gv.sdev(v),
        fmt='.'
    )

def jkCorr(data, bsize=1):
    if bsize==0:
        return data
    else:
        rData = (data.shape[0]//bsize)
        nData = rData*bsize
        dIn   = data[0:nData,:]

        avg   = dIn.mean(axis=0)

        base  = dIn.sum(axis=0) # avg * nData

        dOut  = dIn * 0.0

        for cf in range(rData):
            pvg = dIn[cf*bsize:(cf+1)*bsize,:].sum(axis=0)
            dOut[cf,:] = (base - pvg)/(nData - bsize)

        return dOut[:rData,:]

In the following cell import `FnalHISQMetadata` module giving the correct path

In [44]:
from b2heavy.FnalHISQMetadata import params # change

ens = 'Coarse-1'
mom = '300'
jk  = 11

mdata = params(ens)
Ta,Tb = mdata['hSinks']

heavy = '_k' + mdata['kBStr']
light = '_k' + mdata['kDStr']
qq    = '_m' + mdata['mlStr']

data = h5py.File(f'/Users/pietro/code/data_analysis/BtoD/Alex/Ensembles/FnalHISQ/a0.12/{mdata["hdf5File2"]}')
data = data['data']

$$
 R_{A_1}(\mathbf{p}_\perp,t,T) = \frac{ C_{B_{1S}\rightarrow D^*_a}^{A_j}(\mathbf{p}_\perp,t,T) \,\, C_{D^*_a\rightarrow B_{1S}}^{A_j}(\mathbf{p}_\perp,t,T) }{  C_{D^*_a\rightarrow D^*_{1S}}^{V_4}(\mathbf{0},t,T) \,\,   C_{B_{1S}\rightarrow B_{1S}}^{V_4}(\mathbf{0},t,T) } / \Biggl[\frac{Z_{D^*_a}(\mathbf p_\perp)}{\sqrt{Z_{D^*_a}(0)Z_{D_{1S}^*}(0)}} \frac{{M_{D^*}}^2}{{E^2_{D^*}}(\mathbb p)} e^{-({E_{D^*}}(\mathbb p) - M_{D^*})T}\Biggr]
$$ 

## Reading

in the following cells we read the data from the archive and we build the uncorrected ratio for each smearing and each sink time and we put it in a dictionary `data_read`

In [65]:
data_read = {}
for smr in ['1S','RW']:
    data_read[smr] = {}
    for tsink in [Ta,Tb]:
        print(f'------------- {tsink = } --------- {smr = } -------------------')

        # Numerator 1
        corr = RatioFileString('P5_A2_V2_',tsink,heavy,smr,qq,light,mom) # T sink 1
        num1 = jkCorr(data[corr][:,:Ta+1])
        print(corr)

        # Numerator 2
        num2 = np.flip(num1,axis=-1)

        # Denominator 1
        corr = RatioFileString('V1_V4_V1_',tsink,light,smr,qq,light,'000') # T sink 1
        den1 = jkCorr(data[corr][:,:Ta+1])
        print(corr)

        # Denominator 2
        corr = RatioFileString('P5_V4_P5_',tsink,heavy,smr,qq,heavy,'000') # T sink 1
        den2 = jkCorr(data[corr][:,:Ta+1])
        print(corr)


        data_read[smr][tsink] = num1 * num2 / den1 / den2

------------- tsink = 12 --------- smr = '1S' -------------------
P5_A2_V2_T12_k0.08574_RW_1S_rot_rot_m0.00507_k0.12201_p300
V1_V4_V1_T12_k0.12201_RW_1S_rot_rot_m0.00507_k0.12201_p000
P5_V4_P5_T12_k0.08574_RW_1S_rot_rot_m0.00507_k0.08574_p000
------------- tsink = 13 --------- smr = '1S' -------------------
P5_A2_V2_T13_k0.08574_RW_1S_rot_rot_m0.00507_k0.12201_p300
V1_V4_V1_T13_k0.12201_RW_1S_rot_rot_m0.00507_k0.12201_p000
P5_V4_P5_T13_k0.08574_RW_1S_rot_rot_m0.00507_k0.08574_p000
------------- tsink = 12 --------- smr = 'RW' -------------------
P5_A2_V2_T12_k0.08574_RW_RW_rot_rot_m0.00507_k0.12201_p300
V1_V4_V1_T12_k0.12201_RW_RW_rot_rot_m0.00507_k0.12201_p000
P5_V4_P5_T12_k0.08574_RW_RW_rot_rot_m0.00507_k0.08574_p000
------------- tsink = 13 --------- smr = 'RW' -------------------
P5_A2_V2_T13_k0.08574_RW_RW_rot_rot_m0.00507_k0.12201_p300
V1_V4_V1_T13_k0.12201_RW_RW_rot_rot_m0.00507_k0.12201_p000
P5_V4_P5_T13_k0.08574_RW_RW_rot_rot_m0.00507_k0.08574_p000


In [81]:
for smr in data_read:
    for tsink in data_read[smr]:
        print(f'-------------- {tsink = } {smr = } --------------')
        print(format(data_read[smr][tsink]))

-------------- tsink = 12 smr = '1S' --------------
[0.0401(54) 0.0434(43) 0.0463(44) 0.0484(44) 0.0487(45) 0.0489(46)
 0.0485(47) 0.0489(46) 0.0486(45) 0.0483(44) 0.0462(44) 0.0432(43)
 0.0395(53)]
-------------- tsink = 13 smr = '1S' --------------
[0.0249(30) 0.0219(26) 0.0229(25) 0.0236(26) 0.0240(27) 0.0241(28)
 0.0244(28) 0.0242(28) 0.0241(27) 0.0235(26) 0.0228(25) 0.0218(25)
 0.0222(27)]
-------------- tsink = 12 smr = 'RW' --------------
[0.144(13) 0.1013(77) 0.0988(71) 0.0954(67) 0.0916(65) 0.0934(66)
 0.0950(69) 0.0928(66) 0.0908(64) 0.0946(66) 0.0959(69) 0.0960(73)
 0.109(10)]
-------------- tsink = 13 smr = 'RW' --------------
[0.0846(70) 0.0547(44) 0.0509(40) 0.0478(38) 0.0469(38) 0.0462(39)
 0.0464(40) 0.0462(39) 0.0466(38) 0.0480(38) 0.0500(39) 0.0518(42)
 0.0595(49)]


## Build correction factor

In the following cell we build the correction factor
$$\frac{Z_{D^*_a}(\mathbf p_\perp)}{\sqrt{Z_{D^*_a}(0)Z_{D_{1S}^*}(0)}} \frac{{M_{D^*}}^2}{{E^2_{D^*}}(\mathbf p)}$$

In [68]:
# This cell has only to be evaluated if next python cell is not

from b2heavy.ThreePointFunctions.types3pts import ratio_prerequisites

frm = '/Users/pietro/code/data_analysis/data/QCDNf2p1stag/B2heavy/lattice24_old'
req = ratio_prerequisites(ens,'RA1',mom,readfrom=frm)

Fill the following cells with appropriate values

In [69]:
req = {
    'E0': 1.1795030243540996,
    'm0': 1.0811885828863745,
    'Z0':   {'1S': 4.834848433376765,  'd': 0.04165271597466671},
    'Zpar': {'1S': 1.65041624880138,   'd': 0.03411249733835417},
    'Zbot': {'1S': 1.1715987932859104, 'd': 0.026111483753158932},
}

In [72]:
factor = {
    '1S': req['Zbot']['1S']/np.sqrt( req['Z0']['1S'] * req['Z0']['1S']) * (req['m0']/req['E0'])**2,
    'RW': req['Zbot']['d' ]/np.sqrt( req['Z0']['d' ] * req['Z0']['1S']) * (req['m0']/req['E0'])**2,
}

factor

{'1S': 0.2036108120912836, 'RW': 0.048890360623124775}

here we divide the data that we read by the correction factor

In [82]:
data_corr = {smr: {} for smr in data_read}

for smr in data_read:
    for tsink in data_read[smr]:
        data_corr[smr][tsink] = data_read[smr][tsink] / factor[smr]
        print(f'-------------- {tsink = } {smr = } --------------')
        print(format(data_corr[smr][tsink]))

-------------- tsink = 12 smr = '1S' --------------
[0.197(27) 0.213(21) 0.227(22) 0.238(22) 0.239(22) 0.240(23) 0.238(23)
 0.240(23) 0.238(22) 0.237(22) 0.227(21) 0.212(21) 0.194(26)]
-------------- tsink = 13 smr = '1S' --------------
[0.122(15) 0.108(13) 0.112(12) 0.116(13) 0.118(13) 0.119(14) 0.120(14)
 0.119(14) 0.119(13) 0.116(13) 0.112(12) 0.107(13) 0.109(13)]
-------------- tsink = 12 smr = 'RW' --------------
[2.94(27) 2.07(16) 2.02(14) 1.95(14) 1.87(13) 1.91(14) 1.94(14) 1.90(13)
 1.86(13) 1.93(14) 1.96(14) 1.96(15) 2.23(21)]
-------------- tsink = 13 smr = 'RW' --------------
[1.73(14) 1.119(90) 1.041(82) 0.978(77) 0.958(78) 0.944(80) 0.949(83)
 0.945(80) 0.954(78) 0.981(78) 1.023(81) 1.059(87) 1.22(10)]


## Exponentials + smoothening

$$
\bar R_{A_1}(t,T) = \frac{1}{2}R_{A_1}(t,T) e^{(E-M)T} + \frac{1}{4}R_{A_1}(t,T+1) e^{(E-M)(T+1)} + \frac{1}{4}R_{A_1}(t+1,T+1) e^{(E-M)(T+1)}
$$

In [98]:
exp = np.exp(req['E0']-req['m0'])

barR = {}
barR['1S'] = data_corr['1S'][Ta  ]/2 * exp**Ta     +     data_corr['1S'][Tb]/4 * exp**Tb  +  np.roll(data_corr['1S'][Tb],-1,axis=0)/4 * exp**Tb
barR['RW'] = data_corr['RW'][Ta  ]/2 * exp**Ta     +     data_corr['RW'][Tb]/4 * exp**Tb  +  np.roll(data_corr['RW'][Tb],-1,axis=0)/4 * exp**Tb

In [102]:
for sm in ['1S','RW']:
    print(f'------------ {sm = } -------------')
    print(format(barR[sm]))

------------ sm = '1S' -------------
[0.540(49) 0.540(40) 0.571(40) 0.594(40) 0.601(41) 0.603(42) 0.602(43)
 0.604(42) 0.601(41) 0.593(40) 0.570(40) 0.538(40) 0.512(48)]
------------ sm = 'RW' -------------
[7.88(50) 5.38(29) 5.16(27) 4.93(25) 4.77(24) 4.80(25) 4.86(26) 4.78(25)
 4.73(24) 4.91(25) 5.03(26) 5.09(28) 5.82(38)]
